In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql
!ln -s /var/run/postgresql/.s.PGSQL.5432 /tmp/.s.PGSQL.5432

ln: failed to create symbolic link '/tmp/.s.PGSQL.5432': File exists


In [2]:
conn = psycopg2.connect(database = "datamyne20170707", user = "ram22")
cur = conn.cursor()

In [313]:
cur.execute("select * from familyTreeFLAT where domestic_ultimate_business_name ~* '.*bae.*';")

In [314]:
oneComp = pd.DataFrame(cur.fetchall(), columns = ['duns_number', 'business_name', 'domestic_ultimate_duns_number', 'domestic_ultimate_business_name'])

In [315]:
oneComp

,duns_number,business_name,domestic_ultimate_duns_number,domestic_ultimate_business_name
0,003175072,Bae Systems Norfolk Ship Repair Inc.,876861790,Bae Systems Holdings Inc.
1,004693824,"Bae Systems, Inc.",876861790,Bae Systems Holdings Inc.
2,006481543,Bae Systems Land & Armaments L.P.,876861790,Bae Systems Holdings Inc.
3,040614112,"Bae Systems Land & Armaments, L.P.",876861790,Bae Systems Holdings Inc.
4,040901068,Bae Systems Land & Armaments Inc.,876861790,Bae Systems Holdings Inc.
5,043183123,Bae Systems Controls Inc.,876861790,Bae Systems Holdings Inc.
6,044919884,"Baer's Furniture Co., Inc.",044919884,"Baer's Furniture Co., Inc."
7,050765684,Bae Systems Southeast Shipyards Jacksonville LLC,876861790,Bae Systems Holdings Inc.
8,054456657,"Bae Systems, Inc.",876861790,Bae Systems Holdings Inc.
9,079004032,Bae Systems,079004032,Bae Systems


In [316]:
oneComp[['domestic_ultimate_duns_number', 'domestic_ultimate_business_name']].drop_duplicates().to_csv('bae.csv')

In [317]:
cia = oneComp

In [318]:
field = cia['domestic_ultimate_duns_number'][0]
cur.execute(sql.SQL("""
select consignee_d_b_duns_number, consignee_d_b_business_name, consignee_d_b_country_name, consignee_d_b_domestic_ultimate_duns_number, consignee_d_b_domestic_ultimate_business_name
from billoflading
where consignee_d_b_domestic_ultimate_duns_number ~* '.*%s.*';""" % str(field)))
oneComp = pd.DataFrame(cur.fetchall(), 
                       columns=['consignee_d_b_duns_number', 'consignee_d_b_business_name', 'consignee_d_b_country_name', 'consignee_d_b_domestic_ultimate_duns_number', 'consignee_d_b_domestic_ultimate_business_name'])
for i in range(1, cia.shape[0]):
    field = cia['domestic_ultimate_duns_number'][i]
    cur.execute(sql.SQL("""select consignee_d_b_duns_number, consignee_d_b_business_name, consignee_d_b_country_name, consignee_d_b_domestic_ultimate_duns_number, consignee_d_b_domestic_ultimate_business_name
    from billoflading
    where consignee_d_b_domestic_ultimate_duns_number ~* '.*%s.*';""" % str(field)))
    oneComp = oneComp.append(pd.DataFrame(cur.fetchall(),
                                     columns=['consignee_d_b_duns_number', 'consignee_d_b_business_name', 'consignee_d_b_country_name', 'consignee_d_b_domestic_ultimate_duns_number', 'consignee_d_b_domestic_ultimate_business_name']),
                             ignore_index=True)

In [319]:
oneComp = oneComp[oneComp['consignee_d_b_country_name'] == 'USA']
oneComp.drop_duplicates().to_csv('baeFromBoL.csv', index=False)

In [320]:
oneComp

,consignee_d_b_duns_number,consignee_d_b_business_name,consignee_d_b_country_name,consignee_d_b_domestic_ultimate_duns_number,consignee_d_b_domestic_ultimate_business_name
0,003175072,Bae Systems Norfolk Ship Repair Inc.,USA,876861790,Bae Systems Holdings Inc.
1,043183123,Bae Systems Controls Inc.,USA,876861790,Bae Systems Holdings Inc.
2,043183123,Bae Systems Controls Inc.,USA,876861790,Bae Systems Holdings Inc.
3,043183123,Bae Systems Controls Inc.,USA,876861790,Bae Systems Holdings Inc.
4,043183123,Bae Systems Controls Inc.,USA,876861790,Bae Systems Holdings Inc.
5,043183123,Bae Systems Controls Inc.,USA,876861790,Bae Systems Holdings Inc.
6,043183123,Bae Systems Controls Inc.,USA,876861790,Bae Systems Holdings Inc.
7,043183123,Bae Systems Controls Inc.,USA,876861790,Bae Systems Holdings Inc.
8,043183123,Bae Systems Controls Inc.,USA,876861790,Bae Systems Holdings Inc.
9,043183123,Bae Systems Controls Inc.,USA,876861790,Bae Systems Holdings Inc.


In [321]:
field = cia['domestic_ultimate_duns_number'][0]
cur.execute(sql.SQL("""
select shipper_d_b_duns_number, shipper_d_b_business_name, shipper_d_b_country_name, shipper_d_b_domestic_ultimate_duns_number, shipper_d_b_domestic_ultimate_business_name
from billoflading
where shipper_d_b_domestic_ultimate_duns_number ~* '.*%s.*';""" % str(field)))
oneComp = pd.DataFrame(cur.fetchall(), 
                       columns=['shipper_d_b_duns_number', 'shipper_d_b_business_name', 'shipper_d_b_country_name', 'shipper_d_b_domestic_ultimate_duns_number', 'shipper_d_b_domestic_ultimate_business_name'])
for i in range(1, cia.shape[0]):
    field = cia['domestic_ultimate_duns_number'][i]
    cur.execute(sql.SQL("""select shipper_d_b_duns_number, shipper_d_b_business_name, shipper_d_b_country_name, shipper_d_b_domestic_ultimate_duns_number, shipper_d_b_domestic_ultimate_business_name
    from billoflading
    where shipper_d_b_domestic_ultimate_duns_number ~* '.*%s.*';""" % str(field)))
    oneComp = oneComp.append(pd.DataFrame(cur.fetchall(),
                                     columns=['shipper_d_b_duns_number', 'shipper_d_b_business_name', 'shipper_d_b_country_name', 'shipper_d_b_domestic_ultimate_duns_number', 'shipper_d_b_domestic_ultimate_business_name']),
                             ignore_index=True)

In [322]:
oneComp = oneComp[oneComp['shipper_d_b_country_name'] == 'USA']
oneComp.drop_duplicates().to_csv('baeFromBoLshipper.csv', index=False)

In [323]:
oneComp

,shipper_d_b_duns_number,shipper_d_b_business_name,shipper_d_b_country_name,shipper_d_b_domestic_ultimate_duns_number,shipper_d_b_domestic_ultimate_business_name


In [22]:
conn.commit()
conn.close()

<function connection.close>